In [7]:
import os

In [8]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator\\research'

In [9]:
os.chdir("../")

In [10]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator'

In [31]:
#Update in entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source: str
    local_data_file: Path
    

In [12]:
from src.translator.constants import *
from src.translator.utils.common import read_yaml, create_directories

In [24]:
#Update configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source=config.source,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [25]:
import os
import datasets
import pandas as pd
from src.translator.logging import logger
from src.translator.utils.common import get_size

In [29]:
#Update data ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info("Downloading data")
            dataset = datasets.load_dataset(f'{self.config.source}')

            # Step 2: Convert the dataset to a Pandas DataFrame
            df_train = pd.DataFrame(dataset["train"])
            df_test = pd.DataFrame(dataset["test"])
            df_validation= pd.DataFrame(dataset["validation"])
            
            # Save the DataFrame to a CSV file in the desired folder
            df_train.to_csv(self.config.root_dir + "/train.csv", index=False)
            df_validation.to_csv(self.config.root_dir + "/validation.csv", index=False)
            df_test.to_csv(self.config.root_dir + "/test.csv", index=False)
            
            logger.info("Dataset ingestion succesful")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        

In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2023-08-06 19:36:49,198: INFO: common: yaml file: config\config.yaml loaded successfully]


[2023-08-06 19:36:49,205: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-06 19:36:49,207: INFO: common: created directory at: artifacts]
[2023-08-06 19:36:49,211: INFO: common: created directory at: artifacts/data_ingestion]


Generating validation split: 100%|██████████| 818/818 [00:00<00:00, 2331.84 examples/s]


[2023-08-06 19:38:12,514: INFO: 2404831519: Dataset ingestion succesful]
